In [135]:
import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [136]:
#read the data 
train_data=pd.read_csv('../data/sales_train.csv')
test_data=pd.read_csv('../data/test.csv')
items_data=pd.read_csv('../data/items.csv')
category_data=pd.read_csv('../data/item_categories.csv')
shops_data=pd.read_csv('../data/shops.csv')

In [137]:
train_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [138]:
test_data.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [139]:
#handling outliers
train_data = train_data[train_data.item_price<100000]
train_data = train_data[train_data.item_cnt_day<1000]

In [140]:
#removing negative values in train_data
train_data = train_data[train_data.item_price > 0].reset_index(drop=True)
train_data[train_data.item_cnt_day <= 0].item_cnt_day.unique()
train_data.loc[train_data.item_cnt_day < 1, 'item_cnt_day'] = 0

In [141]:
train_data.loc[train_data.shop_id == 0, 'shop_id'] = 57
test_data.loc[test_data.shop_id == 0, 'shop_id'] = 57
train_data.loc[train_data.shop_id == 1, 'shop_id'] = 58
test_data.loc[test_data.shop_id == 1, 'shop_id'] = 58
train_data.loc[train_data.shop_id == 11, 'shop_id'] = 10
test_data.loc[test_data.shop_id == 11, 'shop_id'] = 10
train_data.loc[train_data.shop_id == 40, 'shop_id'] = 39
test_data.loc[test_data.shop_id == 40, 'shop_id'] = 39

In [142]:
#shops data preprocessing
shops_data.loc[shops_data.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

shops_data['city'] = shops_data['shop_name'].str.split(' ').map(lambda x: x[0])
shops_data['category'] = shops_data['shop_name'].str.split(' ').map(lambda x:x[1]).astype(str)

shops_data.loc[shops_data.city == '!Якутск', 'city'] = 'Якутск'

category = ['Орджоникидзе,', 'ТЦ', 'ТРК', 'ТРЦ','ул.', 'Магазин', 'ТК', 'склад']
shops_data.category = shops_data.category.apply(lambda x: x if (x in category) else 'etc')
shops_data.category.unique()

array(['Орджоникидзе,', 'ТЦ', 'ТРК', 'ТРЦ', 'etc', 'ул.', 'Магазин', 'ТК',
       'склад'], dtype=object)

In [143]:
shops_data.groupby(['category']).sum()

,shop_id
category,
etc,68
Магазин,22
"Орджоникидзе,",57
ТК,184
ТРК,140
ТРЦ,340
ТЦ,883
склад,55
ул.,21


In [144]:
category = ['ТЦ', 'ТРК', 'ТРЦ', 'ТК']
shops_data.category = shops_data.category.apply(lambda x: x if (x in category) else 'etc')
shops_data['shop_city'] = shops_data.city
shops_data['shop_category'] = shops_data.category

shops_data['shop_city'] = LabelEncoder().fit_transform(shops_data['shop_city'])
shops_data['shop_category'] = LabelEncoder().fit_transform(shops_data['shop_category'])

shops_data = shops_data[['shop_id','shop_city', 'shop_category']] 
shops_data.head()

,shop_id,shop_city,shop_category
0,0,29,0
1,1,29,4
2,2,0,4
3,3,1,2
4,4,2,4


In [145]:
#category data preprocessing
category_data['type_code'] = category_data.item_category_name.apply(lambda x: x.split(' ')[0]).astype(str)
category_data.loc[(category_data.type_code == 'Игровые') | (category_data.type_code == 'Аксессуары'), 'category'] = 'Игры'
category_data.loc[category_data.type_code == 'PC', 'category'] = 'Музыка'

category = ['Игры', 'Карты', 'Кино', 'Книги','Музыка', 'Подарки', 'Программы', 'Служебные', 'Чистые']

category_data['type_code'] = category_data.type_code.apply(lambda x: x if (x in category) else 'etc')

print(category_data.groupby(['type_code']).sum())
category_data['type_code'] = LabelEncoder().fit_transform(category_data['type_code'])

category_data['split'] = category_data.item_category_name.apply(lambda x: x.split('-'))
category_data['subtype'] = category_data['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
category_data['subtype_code'] = LabelEncoder().fit_transform(category_data['subtype'])
category_data = category_data[['item_category_id','type_code', 'subtype_code']]

           item_category_id
type_code                  
etc                     236
Игры                    343
Карты                   170
Кино                    195
Книги                   624
Музыка                  345
Подарки                 798
Программы               453
Служебные               159
Чистые                  163


In [146]:
import re
from collections import Counter
from operator import itemgetter
items_data['name_1'], items_data['name_2'] = items_data['item_name'].str.split('[', 1).str
items_data['name_1'], items_data['name_3'] = items_data['item_name'].str.split('(', 1).str

items_data['name_2'] = items_data['name_2'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
items_data['name_3'] = items_data['name_3'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
items_data = items_data.fillna('0')

result_1 = Counter(' '.join(items_data['name_2'].values.tolist()).split(' ')).items()
result_1 = sorted(result_1, key=itemgetter(1))
result_1 = pd.DataFrame(result_1, columns=['feature', 'count'])
result_1 = result_1[(result_1['feature'].str.len() > 1) & (result_1['count'] > 200)]

result_2 = Counter(' '.join(items_data['name_3'].values.tolist()).split(" ")).items()
result_2 = sorted(result_2, key=itemgetter(1))
result_2 = pd.DataFrame(result_2, columns=['feature', 'count'])
result_2 = result_2[(result_2['feature'].str.len() > 1) & (result_2['count'] > 200)]

result = pd.concat([result_1, result_2])
result = result.drop_duplicates(subset=['feature']).reset_index(drop=True)

print('Most common aditional features:', result)

def name_correction(x):
    x = x.lower()
    x = x.partition('[')[0]
    x = x.partition('(')[0]
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x)
    x = x.replace('  ', ' ')
    x = x.strip()
    return x

items_data['item_name'] = items_data['item_name'].apply(lambda x: name_correction(x))
items_data.name_2 = items_data.name_2.apply(lambda x: x[:-1] if x != '0' else '0')

Most common aditional features:          feature  count
0   документация    284
1     английская    340
2        русские    399
3       субтитры    400
4            360    465
5          jewel    552
6           xbox    589
7            ps3    611
8        русская   1428
9       цифровая   1995
10            pc   2585
11        версия   3427
12           box    246
13            3d    409
14           dvd    503
15      digipack    541
16          фирм    757
17           mp3    854
18            cd    871
19        регион   1849
20            bd   2320


In [147]:
items_data['type'] = items_data.name_2.apply(lambda x: x[0:8] if x.split(' ')[0] == 'xbox' else x.split(' ')[0])
items_data.loc[(items_data.type == 'x360') | (items_data.type == 'xbox360'), 'type'] = 'xbox 360'
items_data.loc[items_data.type == '', 'type'] = 'mac'
items_data.type = items_data.type.apply(lambda x: x.replace(' ',''))
items_data.loc[(items_data.type == 'pc') | (items_data.type == 'pс') | (items_data.type == 'рс'), 'type'] = 'pc'
items_data.loc[(items_data.type == 'рs3'), 'type'] = 'ps3'

group_sum = items_data.groupby('type').sum()
group_sum.loc[group_sum.item_category_id < 200]

,item_id,item_category_id
type,,
5c5,12118,16
5c7,26808,32
5f4,7936,7
6dv,7934,7
6jv,7924,7
6l6,7935,7
android,12994,78
hm3,7926,7
j72,31709,28


In [148]:
drop_list = ['5c5', '5c7', '5f4', '6dv', '6jv', '6l6', 'android', 'hm3', 'j72', 'kf6', 'kf7','kg4',
            'ps2', 's3v', 's4v'	,'англ', 'русская', 'только', 'цифро']

items_data.name_2 = items_data.type.apply(lambda x: 'etc' if x in drop_list else x)
items_data = items_data.drop(['type'], axis=1)
items_data.groupby('name_2').sum()

,item_id,item_category_id
name_2,,
0,218310960,872182
etc,253458,550
mac,148743,2323
pc,16991083,106924
ps,431537,1738
ps3,3531772,11422
ps4,895981,3489
psp,692947,2415
xbox360,2362398,10718


In [149]:
items_data['name_2'] = LabelEncoder().fit_transform(items_data['name_2'])
items_data['name_3'] = LabelEncoder().fit_transform(items_data['name_3'])
items_data.drop(['item_name', 'name_1'], axis=1, inplace=True)
items_data.head()

,item_id,item_category_id,name_2,name_3
0,0,40,0,1331
1,1,76,3,42
2,2,40,0,1011
3,3,40,0,1010
4,4,40,0,1572


In [150]:
#join train_data based on item category id etc 
#train_data = train_data.join(items_data, on='item_id', rsuffix='_').join(shops_data, on='shop_id', rsuffix='_').join(category_data, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)


In [151]:
train_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,0.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [152]:
#monthly data preprocessinng
#ts = time.time()
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train_data[train_data.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

In [153]:
train_data['revenue'] = train_data['item_price'] *  train_data['item_cnt_day']

In [154]:
group = train_data.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})

group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=cols, how='left')



matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)#.astype(np.float16))
                                .clip(0,20) # NB clip target here
                                .astype(np.float16))

In [155]:
test_data['date_block_num'] = 34
test_data['date_block_num'] = test_data['date_block_num'].astype(np.int8)
test_data['shop_id'] = test_data['shop_id'].astype(np.int8)
test_data['item_id'] = test_data['item_id'].astype(np.int16)

In [156]:
matrix = pd.concat([matrix, test_data], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True) # 34 month

In [157]:
category_data.columns
matrix = pd.merge(matrix, shops_data, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items_data, on=['item_id'], how='left')
matrix = pd.merge(matrix, category_data, on=['item_category_id'], how='left')
matrix['shop_city'] = matrix['shop_city'].astype(np.int8)
matrix['shop_category'] = matrix['shop_category'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)


In [158]:
"""print(train_data.columns)
group = train_data.groupby(['date_block_num','shop_id','item_id','item_category_id']).agg({'item_price': ['sum','mean']})
group.columns=['item_price_sum','item_mean_sum']
group.reset_index(inplace=True)

#matrix.columns
matrix = pd.merge(matrix, group, on=['date_block_num','shop_id','item_id','item_category_id'], how='left')"""

"print(train_data.columns)\ngroup = train_data.groupby(['date_block_num','shop_id','item_id','item_category_id']).agg({'item_price': ['sum','mean']})\ngroup.columns=['item_price_sum','item_mean_sum']\ngroup.reset_index(inplace=True)\n\n#matrix.columns\nmatrix = pd.merge(matrix, group, on=['date_block_num','shop_id','item_id','item_category_id'], how='left')"

In [159]:
matrix.shape

(11056277, 12)

In [160]:
matrix.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'ID',
       'shop_city', 'shop_category', 'item_category_id', 'name_2', 'name_3',
       'type_code', 'subtype_code'],
      dtype='object')

In [161]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [162]:
matrix = lag_feature(matrix, [1,2,3], 'item_cnt_month')

In [163]:
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)

In [164]:
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3], 'date_item_avg_item_cnt')
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

In [165]:
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)

In [166]:
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)

In [167]:
group = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

In [168]:
group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_item_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
matrix['date_shop_item_avg_item_cnt'] = matrix['date_shop_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_item_avg_item_cnt')
matrix.drop(['date_shop_item_avg_item_cnt'], axis=1, inplace=True)


In [169]:
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix['date_shop_subtype_avg_item_cnt'] = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_subtype_avg_item_cnt')
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)

In [170]:
group = matrix.groupby(['date_block_num', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_city'], how='left')
matrix['date_city_avg_item_cnt'] = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_city_avg_item_cnt')
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)

In [171]:
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'shop_city'], how='left')
matrix['date_item_city_avg_item_cnt'] = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_item_city_avg_item_cnt')
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)

In [172]:
group = train_data.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train_data.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

"""fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)"""

"fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']\nfor i in lags:\n    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]\n    fetures_to_drop += ['delta_price_lag_'+str(i)]\n\nmatrix.drop(fetures_to_drop, axis=1, inplace=True)"

In [173]:
group = train_data.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

In [174]:
matrix['month'] = matrix['date_block_num'] % 12

days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)

In [175]:
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

In [176]:
matrix = matrix[matrix.date_block_num > 3]

In [177]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)

matrix.info()
import gc
gc.collect();

In [178]:
matrix.to_pickle('../data/tot_data_updated.pkl')


In [179]:
matrix.columns 

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'ID',
       'shop_city', 'shop_category', 'item_category_id', 'name_2', 'name_3',
       'type_code', 'subtype_code', 'item_cnt_month_lag_1',
       'item_cnt_month_lag_2', 'item_cnt_month_lag_3',
       'date_avg_item_cnt_lag_1', 'date_item_avg_item_cnt_lag_1',
       'date_item_avg_item_cnt_lag_2', 'date_item_avg_item_cnt_lag_3',
       'date_shop_avg_item_cnt_lag_1', 'date_shop_avg_item_cnt_lag_2',
       'date_shop_avg_item_cnt_lag_3', 'date_cat_avg_item_cnt_lag_1',
       'date_shop_cat_avg_item_cnt_lag_1', 'date_shop_item_avg_item_cnt_lag_1',
       'date_shop_subtype_avg_item_cnt_lag_1', 'date_city_avg_item_cnt_lag_1',
       'date_item_city_avg_item_cnt_lag_1', 'item_avg_item_price',
       'date_item_avg_item_price', 'date_item_avg_item_price_lag_1',
       'date_item_avg_item_price_lag_2', 'date_item_avg_item_price_lag_3',
       'delta_price_lag_1', 'delta_price_lag_2', 'delta_price_lag_3',
       'delta_price